In [2]:
import os
import pandas as pd
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools #homemade module
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [5]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('gtfs_subset/')
stops = gtfs.load_stops('gtfs_subset/')
# no stop times for now due to duplicate keys
# stop_times, tz_sched = gtfs.load_stop_times('gtfs_subset/')
tcal = gtfs.TransitCalendar('gtfs_subset/')
print 'Finished loading GTFS data.'

Finished loading GTFS data.


# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('newdata_parsed.csv')#,parse_dates=dt_columns)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime['Trip'] = bustime['Trip'].str.replace('MTA NYCT_','')
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)

In [3]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('spark_parse/test2.txt',header=None)

# beware!  Bonan is still working on organizing the extract files.  these columns may change.
bustime.columns = ['route','lat','lon','timestamp','vehicle_id','trip_id','trip_date','shape_id','next_stop_id','dist_from_stop','stop_dist_on_trip','presentable_dist']

bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime.set_index(['route','trip_id','trip_date','vehicle_id'],inplace=True,drop=True)#,verify_integrity=True)

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [6]:
# for demonstration, use a subset. Just get data for one line (M5) on one day.
tripDateLookup = "2015-01-06"
bustime = bustime.xs((tripDateLookup),level=(2),drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
print 'Finished loading BusTime data and and slicing one day.'

Finished loading BusTime data and and slicing one day.


In [30]:
bustime.groupby(level=0)['timestamp'].min()

route
MTA NYCT_B1      2015-01-06T01:43:21.000-05:00
MTA NYCT_B11     2015-01-06T04:08:12.000-05:00
MTA NYCT_B12     2015-01-06T01:25:41.000-05:00
MTA NYCT_B13     2015-01-06T04:45:54.000-05:00
MTA NYCT_B14     2015-01-06T01:21:55.000-05:00
MTA NYCT_B15     2015-01-06T01:45:45.927-05:00
MTA NYCT_B16     2015-01-06T05:15:57.000-05:00
MTA NYCT_B17     2015-01-06T01:02:36.000-05:00
MTA NYCT_B2      2015-01-06T04:59:43.000-05:00
MTA NYCT_B20     2015-01-06T04:48:50.000-05:00
MTA NYCT_B24     2015-01-06T05:14:55.000-05:00
MTA NYCT_B25     2015-01-06T00:15:12.000-05:00
MTA NYCT_B26     2015-01-06T01:20:40.000-05:00
MTA NYCT_B3      2015-01-06T00:48:58.000-05:00
MTA NYCT_B31     2015-01-06T04:56:36.000-05:00
MTA NYCT_B32     2015-01-06T08:56:49.805-05:00
MTA NYCT_B35     2015-01-06T00:25:09.335-05:00
MTA NYCT_B36     2015-01-06T00:21:56.439-05:00
MTA NYCT_B37     2015-01-06T04:59:59.000-05:00
MTA NYCT_B38     2015-01-06T00:43:50.000-05:00
MTA NYCT_B39     2015-01-06T08:59:44.000-05:00
MTA NYC

In [7]:
active_services = tcal.get_service_ids('2015-01-06')
trips_ = gtfs.load_trips('gtfs_subset/').reset_index().set_index(['service_id'])
gtfs_trip_ids = trips_.loc[active_services]['trip_id'].unique()

In [8]:
bustime_trip_ids = bustime.index.get_level_values(1).unique()
bustime_trip_ids[:5]

array(['UP_A5-Weekday-SDon-009800_B1_1', 'UP_A5-Weekday-SDon-010000_B1_2',
       'UP_A5-Weekday-SDon-013800_B1_2', 'UP_A5-Weekday-SDon-014000_B1_1',
       'UP_A5-Weekday-SDon-017800_B1_1'], dtype=object)

In [9]:
gtfs_trip_ids[:5]

array(['EN_A5-Weekday-SDon-000000_B82_101',
       'EN_A5-Weekday-SDon-006500_B82_101',
       'EN_A5-Weekday-SDon-014000_B82_101',
       'EN_A5-Weekday-SDon-020500_B82_101',
       'EN_A5-Weekday-SDon-007000_B82_102'], dtype=object)

In [10]:
len(gtfs_trip_ids)

54281

In [11]:
len(bustime_trip_ids)

40553

## Hmmm there appears to be a big difference in the trip_id lists.
Return the non-matching trip_ids and let's examine the data from the GTFS import.

In [12]:
set1 = set(gtfs_trip_ids)
set2 = set(bustime_trip_ids)

unmatched = set1.symmetric_difference(set2) # set(['pig'])

In [21]:
len(unmatched)

13728

In [32]:
trips.loc[unmatched]

,block_id,direction_id,route_id,service_id,shape_id,trip_headsign
trip_id,,,,,,
MQ_A5-Weekday-SDon-102800_M57_11,NaN,1,M72,MQ_A5-Weekday-SDon,M720010,WEST SIDE FREEDOM PL XTOWN
GH_A5-Weekday-SDon-085500_BX402_11,NaN,1,BX40,GH_A5-Weekday-SDon,BX400081,RIVER PK TOWER via TREMONT
QV_A5-Weekday-SDon-089700_Q46_5,NaN,1,Q46,QV_A5-Weekday-SDon,Q460144,KEW GARDENS QUEENS BL via UNION
QV_A5-Weekday-SDon-089700_Q46_4,NaN,0,Q46,QV_A5-Weekday-SDon,Q460239,LK SUCCESS LIJ HOSP via UNION
FP_A5-Weekday-SDon-042400_B52_117,NaN,1,B26,FP_A5-Weekday-SDon,B260122,DNTWN BKLYN TILLARY ST via HALSEY
7713417-LGPA5-LG_A5-Weekday-80-SDon,1494502,0,Q47,LGPA5-LG_A5-Weekday-80-SDon,Q470110,LAGUARDIA AIRPORT TERMINAL A
MQ_A5-Weekday-SDon-045100_M22_308,NaN,1,M22,MQ_A5-Weekday-SDon,M220055,BATTERY PK CTY
CH_A5-Weekday-SDon-090100_MISC_841,NaN,0,S74,CH_A5-Weekday-SDon,S740223,ELTINGVILLE TRANSIT CTR
CS_A5-Weekday-SDon-102700_Q26_1,NaN,1,Q26,CS_A5-Weekday-SDon,Q260092,FLUSHING MAIN ST STA


In [33]:
# of trips by route with no data in BusTime set
pct_missing_by_route = trips.loc[unmatched].groupby('route_id').size()/trips.groupby('route_id').size()
pct_missing_by_route.sort(ascending=False)
pct_missing_by_route

route_id
S86      0.500000
S81      0.333333
S84      0.312500
S94      0.294118
S90      0.277778
S91      0.260000
S98      0.257143
S96      0.232143
S56      0.209402
S92      0.187500
BX20     0.181818
X17A     0.167883
X11      0.163265
Q26      0.145038
S55      0.142857
X3       0.138889
X2       0.134146
X19      0.130435
S54      0.129518
S42      0.129353
X21      0.119048
M98      0.118280
S89      0.114865
Q77      0.113943
X4       0.113924
X68      0.111111
X17J     0.110132
S93      0.107042
X9       0.106870
X37      0.105691
           ...   
QM1      0.007752
BXM7     0.007533
QM24     0.007299
Q37      0.006605
BM2      0.006494
Q66      0.005271
BXM6     0.004926
QM20     0.004717
QM4      0.004016
Q65      0.003864
QM6      0.003717
Q21      0.002941
BXM9     0.002762
Q53      0.002294
BM4           NaN
BXM18         NaN
Q104          NaN
Q35           NaN
Q70           NaN
QM10          NaN
QM12          NaN
QM15          NaN
QM16          NaN
QM17          NaN
Q